In [2]:
import argparse
import os
import random
import torch
import yaml
from torch.utils.data import DataLoader
from train_model.dataset_utils import prepare_test_data_set, prepare_eval_data_set, prepare_train_data_set
import numpy as np
from os import listdir
from dataset_utils import text_processing
from train_model.helper import run_model, build_model
from IPython.display import Image, display, clear_output
import matplotlib.pyplot as plt
import cv2
import json
from random import shuffle

# Get test images
# wget http://images.cocodataset.org/zips/test2015.zip, unzip and update path to image directory
im_dir = '/private/home/nvivek/vizwiz/Images/'
# Get the models
# wget https://s3-us-west-1.amazonaws.com/vqa-suite/pretrained_models/most_data_models.tar.gz and 
# move the best_model.pth from the uncompressed file to the folder best_model
# Get features
# mkdir data
# cd data
# wget https://s3-us-west-1.amazonaws.com/vqa-suite/data/vqa2.0_glove.6B.300d.txt.npy
# wget https://s3-us-west-1.amazonaws.com/vqa-suite/data/vocabulary_vqa.txt
# wget https://s3-us-west-1.amazonaws.com/vqa-suite/data/answers_vqa.txt
# wget https://s3-us-west-1.amazonaws.com/vqa-suite/data/imdb.tar.gz
# wget https://s3-us-west-1.amazonaws.com/vqa-suite/data/rcnn_10_100.tar.gz
# wget https://s3-us-west-1.amazonaws.com/vqa-suite/data/large_vocabulary_vqa.txt
# wget https://s3-us-west-1.amazonaws.com/vqa-suite/data/large_vqa2.0_glove.6B.300d.txt.npy
# tar -xf rcnn_10_100.tar.gz
# tar -xf imdb.tar.gz

config_file = '/checkpoint/nvivek/test_large_answers_copy_keep_score/results/baseline_vizwiz_predict/5772/config.yaml'
model_file = '/checkpoint/nvivek/test_large_answers_copy_keep_score/results/baseline_vizwiz_predict/5772/best_model.pth'
# model_file = '/checkpoint/nvivek/ft_3_class_soft_score_sweep_7k_answer_fasttext/results/baseline_vizwiz_predict/data.batch_size.128_model.ans_loss_weight.0.1_att_loss_weight.0.1_itf_lr.0.005_optimizer.method.Adamax_par.lr.0.005_84615/best_model.pth'
im_feat_dir = '/private/home/nvivek/VQA/training_data/rcnn_adaptive_vizwiz/vizwiz_ocr_text'
test_annot_file = '/private/home/nvivek/vizwiz/Annotations/v2_mscoco_vizwiz_test_annotations.json'
train_annot_file = '/private/home/nvivek/VQA/training_data/v2_mscoco_vizwiz_train_annotations.json'
val_annot_file = '/private/home/nvivek/VQA/training_data/v2_mscoco_vizwiz_val_annotations.json'
max_loc = 137
att_sup_file = 'attention_sup.json'

with open(att_sup_file, 'r') as f:
    att_sups = json.load(f)

def visualize(im_file, im_feature):
    im = cv2.imread(im_file)
    im_id = int(im_file.split('/')[-1].split('.')[0].split('_')[-1])
    im = im[:, :, (2, 1, 0)]
    fig, ax = plt.subplots(figsize=(12, 12))
    ax.imshow(im)

    im_data = np.load(im_feature)
    text = im_data.item().get('image_text').tolist()
    bbox = im_data.item().get('image_bboxes').tolist()
    bbox_source = im_data.item().get('image_bbox_source').tolist()

    if im_id >= 20000 and im_id < 28000:
        att_sup = [0]*len(bbox_source)
    else:
        att_sup = att_sups[str(im_id)]  
    
    for i,(source,bbox,text) in enumerate(zip(bbox_source, bbox, text)):
        
        if att_sup[i] >= 0.1:
            text += ' attend here'
            print(i)
            
        edgecolor = 'red'
        if source[0] == 1:
            edgecolor = 'green'
        ax.add_patch(
            plt.Rectangle((bbox[0], bbox[1]),
                          bbox[2] - bbox[0],
                          bbox[3] - bbox[1], fill=False, alpha=0.5,
                          edgecolor=edgecolor, linewidth=3.5)
            )
        box_text = '{:s}'.format(text)
        ax.text(bbox[0] + 5, bbox[1] + 5,
                box_text,
                bbox=dict(alpha=0.5),
                fontsize=14, color='white')
    plt.axis('off')
    plt.tight_layout()
    plt.draw()
    plt_file_path = im_file.split('/')[-1].replace(".jpg", "_pltbb.jpg")
    plt.savefig(plt_file_path)
    clear_output()
    display(Image(filename=plt_file_path))

def visualize_att(im_file, im_feature, att, k, show=False):
    im = cv2.imread(im_file)
    im = im[:, :, (2, 1, 0)]
    if show:
        fig, ax = plt.subplots(figsize=(12, 12))
        ax.imshow(im)

    im_data = np.load(im_feature)
    text = im_data.item().get('image_text').tolist()
    bbox = im_data.item().get('image_bboxes').tolist()
    bbox_source = im_data.item().get('image_bbox_source').tolist()

    top_k_idx = list(np.argsort(att)[-k:])
    best_text = ''

    for i,(source,bbox,text) in enumerate(zip(bbox_source, bbox, text)):

        edgecolor = 'red'
        if source[0] == 1:
            edgecolor = 'green'
        fill = False
        alpha = 0.5
        if i not in top_k_idx:
            continue
        if i == top_k_idx[-1]:
            best_text = text
        if not show:
            continue
        ax.add_patch(
            plt.Rectangle((bbox[0], bbox[1]),
                          bbox[2] - bbox[0],
                          bbox[3] - bbox[1], fill=fill, alpha=alpha,
                          edgecolor=edgecolor, linewidth=3.5)
            )
        box_text = '{:s} {:.3f}'.format(text, att[i])
        
        ax.text(bbox[0] + 5, bbox[1] + 5,
                box_text,
                bbox=dict(alpha=0.5),
                fontsize=14, color='white')

    if show:
        plt.axis('off')
        plt.tight_layout()
        plt.draw()
        plt_file_path = im_file.split('/')[-1].replace(".jpg", "_pltatt.jpg")
        plt.savefig(plt_file_path)
        clear_output()
        display(Image(filename=plt_file_path))
    return best_text
    

def get_image(source='test'):
    im_files = [f for f in os.listdir(im_dir) if source in f]
    im_file = random.choice(im_files)
    im_path = os.path.join(im_dir, im_file)
    clear_output()
    print(im_path)
    im_feature_path = os.path.join(im_feat_dir, 'COCO_' + im_file.split('/')[-1].replace(".jpg", ".npy").lower())
    # visualize(im_path, im_feature_path)
    # display(Image(filename=im_path))
    return im_file

def get_imdb(im_file, question_str):
    imdb = []
    imdb.append({'dataset_name': 'vizwiz', 'version': 1, 'has_answer': False, 'has_gt_layout': False})
    iminfo = {}
    iminfo['image_name'] = im_file.replace('.jpg', '')
    iminfo['img_id'] = int(iminfo['image_name'].split('_')[-1])
    iminfo['question_id'] = 0
    iminfo['feature_path'] = 'COCO_' + iminfo['image_name'].lower() + '.npy'
    iminfo['question_str'] = question_str
    iminfo['question_tokens'] = text_processing.tokenize(iminfo['question_str'])
    imdb.append(iminfo)
    return imdb

def print_result(question_ids, soft_max_result, ans_dic):
    predicted_answers = np.argsort(soft_max_result, axis=1)[0][-3:]
    # predicted_answers = np.argmax(soft_max_result, axis=1)
    answers = []
    for idx, pred_idx in enumerate(predicted_answers):
        pred_ans = ans_dic.idx2word(pred_idx)
        answers.append(pred_ans)
    return answers

with open(config_file, 'r') as f:
    config = yaml.load(f)

with open(test_annot_file, 'r') as f:
    data = json.load(f)
test_annot = data['annotations']

with open(train_annot_file, 'r') as f:
    data = json.load(f)
train_annot = data['annotations']

with open(val_annot_file, 'r') as f:
    data = json.load(f)
val_annot = data['annotations']

data_set_test = prepare_test_data_set(**config['data'], **config['model'], verbose=True, test_mode=True)
data_set_val = prepare_eval_data_set(**config['data'], **config['model'], verbose=True, test_mode=True)
data_set_train = prepare_train_data_set(**config['data'], **config['model'], verbose=True, test_mode=True)

myModel = build_model(config, data_set_test)
state_dict = torch.load(model_file)['state_dict']
from collections import OrderedDict
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k.replace('module.','')
    new_state_dict[name] = v

myModel.load_state_dict(new_state_dict)

i = 0

results = '/checkpoint/nvivek/test_large_answers_copy_keep_score/results/baseline_vizwiz_predict/5772/best_model_predict_test.json'
with open(results, 'r') as f:
    predictions = json.load(f)

im_dir = '/private/home/nvivek/vizwiz/Images'

test_questions_file = '/private/home/nvivek/VQA/training_data/v2_OpenEnded_mscoco_vizwiz_test_questions.json'
val_questions_file = '/private/home/nvivek/VQA/training_data/v2_OpenEnded_mscoco_vizwiz_val_questions.json'

with open(test_questions_file, 'r') as f:
    test_questions = json.load(f)

with open(val_questions_file, 'r') as f:
    val_questions = json.load(f)

val_questions = val_questions['questions']
test_questions = test_questions['questions']

def isInt(s):
    try: 
        int(s)
        return True
    except ValueError:
        return False

vocabulary_100k.txt
imdb does not contain ground-truth layout
Loading model and config ...
vocabulary_100k.txt
imdb does not contain ground-truth layout
Loading model and config ...
vocabulary_100k.txt
imdb does not contain ground-truth layout
Loading model and config ...
Vocab 75501
Answers 7371
[{'method': 'default_image', 'par': {}}]
Building final model
ModuleList(
  (0): AttQuestionEmbedding(
    (embedding): Embedding(75501, 300)
    (LSTM): LSTM(300, 1024, batch_first=True)
    (Dropout): Dropout(p=0.0)
    (conv1): Conv1d(1024, 512, kernel_size=(1,), stride=(1,))
    (conv2): Conv1d(512, 2, kernel_size=(1,), stride=(1,))
  )
)
ModuleList(
  (0): ModuleList(
    (0): image_embedding(
      (image_attention_model): top_down_attention(
        (modal_combine): non_linear_elmt_multiply(
          (Fa_image): nonlinear_layer(
            (main): Sequential(
              (0): Linear(in_features=2048, out_features=5000, bias=True)
              (1): ReLU()
            )
          )
 

In [3]:
# Good examples
# VizWiz_train_000000009017.jpg
# VizWiz_train_000000013282.jpg
# VizWiz_train_000000011531.jpg
# VizWiz_train_000000007658.jpg
# VizWiz_train_000000008618.jpg
# VizWiz_train_000000010968.jpg
# VizWiz_train_000000010978.jpg

# VizWiz_test_000000020573.jpg
# VizWiz_test_000000020188.jpg

# Tries to read
# VizWiz_test_000000026706.jpg
# VizWiz_test_000000026163.jpg - answer not in vocab
# VizWiz_test_000000025030.jpg
# VizWiz_test_000000024759.jpg
# VizWiz_test_000000026580.jpg
# VizWiz_test_000000023290.jpg
# VizWiz_test_000000026569.jpg
# VizWiz_test_000000020277.jpg
# VizWiz_test_000000021706.jpg
# VizWiz_test_000000025870.jpg


# More examples
# VizWiz_test_000000025557.jpg decaf
# VizWiz_test_000000023489.jpg garlic
# VizWiz_test_000000027300.jpg hp
# VizWiz_test_000000025822.jpg good - sauvignon
# VizWiz_test_000000022343.jpg good - beans
# VizWiz_test_000000022956.jpg - era laundry
# VizWiz_test_000000022974.jpg - sweet
# VizWiz_test_000000024612.jpg - tea flavors left to right
# VizWiz_test_000000020566.jpg - bacon for cheetos
# VizWiz_test_000000027822.jpg - better to pick unsuitable/unasnwerable

examples = ['VizWiz_train_000000009017.jpg', 'VizWiz_train_000000013282.jpg', 'VizWiz_train_000000011531.jpg', 
            'VizWiz_train_000000007658.jpg', 'VizWiz_train_000000008618.jpg', 'VizWiz_train_000000010968.jpg', 
            'VizWiz_train_000000010978.jpg']

source = 'test'
train_index = 0
val_index = 28000
test_index = 20000
# im_file = 'VizWiz_train_000000019050.jpg'
# im_file = 'VizWiz_train_000000015775.jpg'
# im_file = 'VizWiz_train_000000019377.jpg'
# im_file = 'VizWiz_test_000000022465.jpg'
# im_file = get_image(source)
# im_file = 'VizWiz_test_000000026163.jpg'
# im_file = examples[i%len(examples)]
# i+=1
# im_file_path = os.path.join(im_dir, im_file)
# im_feature_path = os.path.join(im_feat_dir, 'COCO_' + im_file.split('/')[-1].replace(".jpg", ".npy").lower())

# print(im_file)
# im_id = int(im_file.split('.')[0].split('_')[-1])

# question_str = ''
# gt = '' 
# if source == 'train':
#     question_str = train_annot[im_id - train_index]['question']
#     gt = train_annot[im_id - train_index]['multiple_choice_answer']
# elif source == 'val':
#     question_str = val_annot[im_id - vsl_index]['question']
#     gt = val_annot[im_id - val_index]['multiple_choice_answer']
# elif source == 'test':
#     question_str = test_annot[im_id - test_index]['question']

# data_set_test.datasets[0].imdb = get_imdb(im_file, question_str)
# data_reader_test = DataLoader(data_set_test, shuffle=False, batch_size=1)
ans_dic = data_set_test.answer_dict

# myModel.eval()
# question_ids, soft_max_result, i_att, it_att = run_model(myModel, data_reader_test, ans_dic.UNK_idx)
total = 0
shuffle(predictions)
modified_pred = []
for prediction in predictions:
    # clear_output()
    im_file = prediction['image']
    im_file_path = os.path.join(im_dir, im_file)
    im_feature_path = os.path.join(im_feat_dir, 'COCO_' + im_file.split('/')[-1].replace(".jpg", ".npy").lower())
    im_id = int(im_file.replace('.jpg','').split('_')[-1])
    print(im_id)
    question = test_questions[im_id - test_index]['question']
    answer = prediction['answer']
    # gt = val_annot[im_id - val_index]['multiple_choice_answer']
    # all_answers = [ans['answer'] for ans in test_annot[im_id - test_index]['answers']]
    data_set_test.datasets[0].imdb = get_imdb(im_file, question)
    data_reader_test = DataLoader(data_set_test, shuffle=False, batch_size=1)
    myModel.eval()
    question_ids, soft_max_result, i_att, it_att = run_model(myModel, data_reader_test, ans_dic.UNK_idx)
    predicted_answers = print_result(question_ids, soft_max_result, ans_dic)
    butd_att = i_att[0].cpu().detach().numpy().reshape((137,)).tolist() 
    text_att = it_att[0].cpu().detach().numpy().reshape((137,)).tolist()
    best_text = visualize_att(im_file_path, im_feature_path, text_att, 5)        
    
    total += 1
    print(total)
    if answer == '<copy>':
#         visualize(im_file_path, im_feature_path)
#         command = input()
#         visualize_att(im_file_path, im_feature_path, text_att, 5, show=True)
        print(im_file)
        print(question)
        # print("Groundtruth: " + gt)
        # print(all_answers)
        # print("Answer: " + predicted_answers[-1])
        # print("Image text " + best_text)
        # print("Top answers " + str(predicted_answers))
        new_prediction = {}
        new_prediction['image'] = im_file
        try:
            best_text.encode('ascii')
            new_prediction['answer'] = best_text
        except:
            new_prediction['answer'] = predicted_answers[-2]
        modified_pred.append(new_prediction)
        # print(sorted(text_att))
        # print(sorted(butd_att))
#         command = input()
#         if command == 'bye':
#             break
#         else:
#             continue
    else:
        modified_pred.append(prediction)

        
# with open('copy_model_predict_test.json', 'w') as f:
#     json.dumps(modified_pred)
# visualize(im_file_path, im_feature_path)
# print(question_str)
# print("Groundtruth: " + gt)
# print("Answer: ")
# print_result(question_ids, soft_max_result, ans_dic)

# visualize_att(im_file_path, im_feature_path, butd_att,5)
# visualize_att(im_file_path, im_feature_path, text_att,5)
# print(sum(butd_att))
# print(sum(text_att))
# print(sorted(butd_att))
# print(sorted(text_att))
# print(len([x for x in butd_att if x > 0]))
# print(len([x for x in text_att if x > 0]))

21715
Before eval
Model size 2.833299784 GB
1
21136
Before eval
Model size 2.833301664 GB
2
27432
Before eval
Model size 2.833301664 GB
3
25830
Before eval
Model size 2.833301664 GB
4
23033
Before eval
Model size 2.833301664 GB
5
26413
Before eval
Model size 2.833301664 GB
6
24712
Before eval
Model size 2.833301664 GB
7
27005
Before eval
Model size 2.833301664 GB
8
VizWiz_test_000000027005.jpg
What dinner is this?
25259
Before eval
Model size 2.833301664 GB
9
24045
Before eval
Model size 2.833301664 GB
10
VizWiz_test_000000024045.jpg
What flavor is this?
22300
Before eval
Model size 2.833301664 GB
11
21436
Before eval
Model size 2.833301664 GB
12
VizWiz_test_000000021436.jpg
Help me identify these bills, please.
22637
Before eval
Model size 2.833301664 GB
13
VizWiz_test_000000022637.jpg
Can you tell me what the temperature is set at, please?
22256
Before eval
Model size 2.833301664 GB
14
24564
Before eval
Model size 2.833301664 GB
15
21727
Before eval
Model size 2.833301664 GB
16
26914

130
26189
Before eval
Model size 2.833301664 GB
131
VizWiz_test_000000026189.jpg
what is this
20647
Before eval
Model size 2.833301664 GB
132
VizWiz_test_000000020647.jpg
Could you please tell me about where this doll is sitting?
20128
Before eval
Model size 2.833301664 GB
133
VizWiz_test_000000020128.jpg
What is this a can of?
20202
Before eval
Model size 2.833301664 GB
134
26425
Before eval
Model size 2.833301664 GB
135
21928
Before eval
Model size 2.833301664 GB
136
27941
Before eval
Model size 2.833301664 GB
137
VizWiz_test_000000027941.jpg
What's in this bottle?
25221
Before eval
Model size 2.833301664 GB
138
22108
Before eval
Model size 2.833301664 GB
139
24609
Before eval
Model size 2.833301664 GB
140
VizWiz_test_000000024609.jpg
What food is this?
26297
Before eval
Model size 2.833301664 GB
141
VizWiz_test_000000026297.jpg
How many keys does the keyboard have?
23812
Before eval
Model size 2.833301664 GB
142
VizWiz_test_000000023812.jpg
What is this spice?
26181
Before eval
Mode

Model size 2.833301664 GB
249
VizWiz_test_000000026802.jpg
What is this? 
27574
Before eval
Model size 2.833301664 GB
250
26517
Before eval
Model size 2.833301664 GB
251
VizWiz_test_000000026517.jpg
What's in this box?
21074
Before eval
Model size 2.833301664 GB
252
24921
Before eval
Model size 2.833301664 GB
253
27141
Before eval
Model size 2.833301664 GB
254
26853
Before eval
Model size 2.833301664 GB
255
VizWiz_test_000000026853.jpg
What does the black button on the top left say?
21687
Before eval
Model size 2.833301664 GB
256
23048
Before eval
Model size 2.833301664 GB
257
25918
Before eval
Model size 2.833301664 GB
258
VizWiz_test_000000025918.jpg
Can you tell what's in this can please?
21616
Before eval
Model size 2.833301664 GB
259
21892
Before eval
Model size 2.833301664 GB
260
VizWiz_test_000000021892.jpg
What is the box title?
27728
Before eval
Model size 2.833301664 GB
261
21070
Before eval
Model size 2.833301664 GB
262
VizWiz_test_000000021070.jpg
What is this?
22973
Before

Model size 2.833301664 GB
370
24331
Before eval
Model size 2.833301664 GB
371
26428
Before eval
Model size 2.833301664 GB
372
VizWiz_test_000000026428.jpg
Hi, I know the print is small but can you read what color this outfit is on the label I just took a picture of? Thank you.
24192
Before eval
Model size 2.833301664 GB
373
VizWiz_test_000000024192.jpg
What is it?
20614
Before eval
Model size 2.833301664 GB
374
20342
Before eval
Model size 2.833301664 GB
375
VizWiz_test_000000020342.jpg
What is this?
24747
Before eval
Model size 2.833301664 GB
376
VizWiz_test_000000024747.jpg
What color is it?
25566
Before eval
Model size 2.833301664 GB
377
23815
Before eval
Model size 2.833301664 GB
378
24850
Before eval
Model size 2.833301664 GB
379
27131
Before eval
Model size 2.833301664 GB
380
25088
Before eval
Model size 2.833301664 GB
381
25315
Before eval
Model size 2.833301664 GB
382
VizWiz_test_000000025315.jpg
Yes, I need to determine the two words in this visual Captcha image.  Thank you ve

Model size 2.833301664 GB
489
27385
Before eval
Model size 2.833301664 GB
490
VizWiz_test_000000027385.jpg
What is in this can?
25334
Before eval
Model size 2.833301664 GB
491
VizWiz_test_000000025334.jpg
what is this?
26944
Before eval
Model size 2.833301664 GB
492
VizWiz_test_000000026944.jpg
What is this?
27120
Before eval
Model size 2.833301664 GB
493
25061
Before eval
Model size 2.833301664 GB
494
25083
Before eval
Model size 2.833301664 GB
495
26636
Before eval
Model size 2.833301664 GB
496
VizWiz_test_000000026636.jpg
What is this?
21020
Before eval
Model size 2.833301664 GB
497
VizWiz_test_000000021020.jpg
What is this?
22069
Before eval
Model size 2.833301664 GB
498
22570
Before eval
Model size 2.833301664 GB
499
26342
Before eval
Model size 2.833301664 GB
500
20888
Before eval
Model size 2.833301664 GB
501
VizWiz_test_000000020888.jpg
What color and pattern are these shorts? 
24448
Before eval
Model size 2.833301664 GB
502
21584
Before eval
Model size 2.833301664 GB
503
20795

...

Model size 2.833301664 GB
7458
25966
Before eval
Model size 2.833301664 GB
7459
25949
Before eval
Model size 2.833301664 GB
7460
VizWiz_test_000000025949.jpg
What type of spice is this?
26662
Before eval
Model size 2.833301664 GB
7461
27576
Before eval
Model size 2.833301664 GB
7462
23890
Before eval
Model size 2.833301664 GB
7463
27304
Before eval
Model size 2.833301664 GB
7464
VizWiz_test_000000027304.jpg
What is this a picture of?
21044
Before eval
Model size 2.833301664 GB
7465
VizWiz_test_000000021044.jpg
What type of coffee is this?
20942
Before eval
Model size 2.833301664 GB
7466
20899
Before eval
Model size 2.833301664 GB
7467
23806
Before eval
Model size 2.833301664 GB
7468
VizWiz_test_000000023806.jpg
Please, I know this is Healthy Choice, but can you tell me what type of frozen dinner this is? Thanks.
20288
Before eval
Model size 2.833301664 GB
7469
VizWiz_test_000000020288.jpg
Can you see if this is for acid loving plants or for roses? Thank you
23841
Before eval
Model size

Model size 2.833301664 GB
7578
23401
Before eval
Model size 2.833301664 GB
7579
20416
Before eval
Model size 2.833301664 GB
7580
22172
Before eval
Model size 2.833301664 GB
7581
22809
Before eval
Model size 2.833301664 GB
7582
21095
Before eval
Model size 2.833301664 GB
7583
VizWiz_test_000000021095.jpg
What kind of soup is this?
21373
Before eval
Model size 2.833301664 GB
7584
VizWiz_test_000000021373.jpg
What page is this? Thank you.
24250
Before eval
Model size 2.833301664 GB
7585
VizWiz_test_000000024250.jpg
What's the breed of my dog?
25709
Before eval
Model size 2.833301664 GB
7586
21086
Before eval
Model size 2.833301664 GB
7587
27080
Before eval
Model size 2.833301664 GB
7588
26447
Before eval
Model size 2.833301664 GB
7589
20829
Before eval
Model size 2.833301664 GB
7590
20024
Before eval
Model size 2.833301664 GB
7591
21117
Before eval
Model size 2.833301664 GB
7592
VizWiz_test_000000021117.jpg
Will you tell me what this bottle says?
21783
Before eval
Model size 2.833301664 G

Model size 2.833301664 GB
7700
VizWiz_test_000000026441.jpg
What is in this can?
24410
Before eval
Model size 2.833301664 GB
7701
VizWiz_test_000000024410.jpg
What's in this bottle
21882
Before eval
Model size 2.833301664 GB
7702
22458
Before eval
Model size 2.833301664 GB
7703
23530
Before eval
Model size 2.833301664 GB
7704
26716
Before eval
Model size 2.833301664 GB
7705
VizWiz_test_000000026716.jpg
What's on the breast of this collared shirt? 
25573
Before eval
Model size 2.833301664 GB
7706
24113
Before eval
Model size 2.833301664 GB
7707
VizWiz_test_000000024113.jpg
What's this?
24672
Before eval
Model size 2.833301664 GB
7708
25858
Before eval
Model size 2.833301664 GB
7709
VizWiz_test_000000025858.jpg
What does the label on this box say?
24941
Before eval
Model size 2.833301664 GB
7710
VizWiz_test_000000024941.jpg
What is this item?
20932
Before eval
Model size 2.833301664 GB
7711
VizWiz_test_000000020932.jpg
What is this?
25439
Before eval
Model size 2.833301664 GB
7712
22940


Model size 2.833301664 GB
7821
22113
Before eval
Model size 2.833301664 GB
7822
VizWiz_test_000000022113.jpg
What is this can?
21275
Before eval
Model size 2.833301664 GB
7823
25751
Before eval
Model size 2.833301664 GB
7824
VizWiz_test_000000025751.jpg
What exactly is this design?
22302
Before eval
Model size 2.833301664 GB
7825
VizWiz_test_000000022302.jpg
what kind of candy is this?
27041
Before eval
Model size 2.833301664 GB
7826
22994
Before eval
Model size 2.833301664 GB
7827
25348
Before eval
Model size 2.833301664 GB
7828
VizWiz_test_000000025348.jpg
What is the title of this catalog?
24148
Before eval
Model size 2.833301664 GB
7829
VizWiz_test_000000024148.jpg
Is it good or not?
23109
Before eval
Model size 2.833301664 GB
7830
22899
Before eval
Model size 2.833301664 GB
7831
VizWiz_test_000000022899.jpg
What is this?
20663
Before eval
Model size 2.833301664 GB
7832
20894
Before eval
Model size 2.833301664 GB
7833
22839
Before eval
Model size 2.833301664 GB
7834
20137
Before ev

Model size 2.833301664 GB
7937
26533
Before eval
Model size 2.833301664 GB
7938
23197
Before eval
Model size 2.833301664 GB
7939
26275
Before eval
Model size 2.833301664 GB
7940
20994
Before eval
Model size 2.833301664 GB
7941
22217
Before eval
Model size 2.833301664 GB
7942
22283
Before eval
Model size 2.833301664 GB
7943
24217
Before eval
Model size 2.833301664 GB
7944
26121
Before eval
Model size 2.833301664 GB
7945
VizWiz_test_000000026121.jpg
What does this say.
26992
Before eval
Model size 2.833301664 GB
7946
VizWiz_test_000000026992.jpg
What color is the cloth and the necklace?
23368
Before eval
Model size 2.833301664 GB
7947
VizWiz_test_000000023368.jpg
What flavor dip is this?
26773
Before eval
Model size 2.833301664 GB
7948
VizWiz_test_000000026773.jpg
What is this?
21697
Before eval
Model size 2.833301664 GB
7949
20206
Before eval
Model size 2.833301664 GB
7950
VizWiz_test_000000020206.jpg
Nothing happened. Just a minute. What is that?
20387
Before eval
Model size 2.83330166

In [5]:
print(len(modified_pred))
print(modified_pred[0])

8000
{'image': 'VizWiz_test_000000021715.jpg', 'answer': 'yes'}


In [6]:
with open('copy_model_predict_test_try2.json', 'w') as f:
    json.dump(modified_pred, f)
f.close()